# Purpose: Phuong BeV Microglia OGD Cell Analysis

### Purpose: To quantify cell features of already segmented microglia images from step 1 in this folder

Created by: Hawley Helmbrecht

Creation Date: 06/4/2021 (Original file)

Edit Date: 10/27/2021 - Change the file to be for Phuongs BeV Analysis rather than the Australian Pig Project

*Step 1: Import Necessary Packages*

In [1]:
import numpy as np
import pandas as pd
from scipy import ndimage

import skimage.filters
from skimage import morphology
from skimage.measure import label, regionprops, regionprops_table
from skimage.color import label2rgb
from skimage import io
from skimage import measure 

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import watermark
import os
from PIL import Image

*Step 2: User Inputs*

In [2]:
#replace the example path from my computer with the path to the image on your computer

cell_folder = '/Users/nelsschimek/Documents/nancelab/Data/caffeine/li_thresh'

file_type = '.npy'

*Step 3: Defining a Folder Cleaner Function to only Return Tif Images*

In [3]:
def folder_cleaner(folder, image_type):
    k=0
    for files in folder:
        if image_type in str(files):
            k+=1
        else:
            folder = np.delete(folder, np.argwhere(folder == str(files)))
    return folder

*Step 4: Get All Images in the Folder*

In [4]:
arr = os.listdir(cell_folder)
file_list = np.asarray(arr)
file_list = folder_cleaner(file_list, file_type)

In [5]:
file_list

array(['40x_Plate8_Slice130_ScWM_Image3_100724_li_thresh.npy',
       '40x_Plate4_Slice80_CC_Image1_093024_li_thresh.npy',
       '40x_Plate11_Slice160_ScWM_Image3_102224_li_thresh.npy',
       '40x_Plate8_Slice31_Ctx_Image1_100724_li_thresh.npy',
       '40x_Plate8_Slice136_CC_Image1_100724_li_thresh.npy',
       '40x_Plate5_Slice85_Th_Image1_100224_li_thresh.npy',
       '40x_Plate8_Slice31_Hipp_Image3_100724_li_thresh.npy',
       '40x_Plate7_Slice128_ScWM_Image3_101924_li_thresh.npy',
       '40x_Plate5_Slice86_CC_Image2_100224_li_thresh.npy',
       '40x_Plate6_Slice117_Ctx_Image3_100324_li_thresh.npy',
       '40x_Plate1_Slice15_BG_Image3_080524_li_thresh.npy',
       '40x_Plate9_Slice204_CC_Image2_101024_li_thresh.npy',
       '40x_Plate12_Slice166_ScWM_Image2_102224_li_thresh.npy',
       '40x_Plate4_Slice80_BG_Image3_093024_li_thresh.npy',
       '40x_Plate5_Slice87_Ctx_Image2_100224_li_thresh.npy',
       '40x_Plate7_Slice122_Ctx_Image3_101824_li_thresh.npy',
       '40x_Plat

*Step 5: Segmenting and Calculating Region Features on All Images*

In [6]:
properties_list = ('area', 'bbox_area', 'centroid', 'convex_area', 
                   'eccentricity', 'equivalent_diameter', 'euler_number', 
                   'extent', 'filled_area', 'major_axis_length', 
                   'minor_axis_length', 'orientation', 'perimeter', 'solidity')


In [8]:
j = 0
for names in file_list:
        file_name = str(cell_folder + '/' + names)
        new_binary_li = np.load(file_name)
        label_image = label(new_binary_li)
        
        #Feel free to add them here as well. The computational time is pretty efficient
        props = measure.regionprops_table(label_image, properties=(properties_list))

        if j == 0:
            df = pd.DataFrame(props)
            df['filename'] = names
        else:
            df2 = pd.DataFrame(props)
            df2['filename'] = names
            df = pd.concat([df, df2])

        j = 1

In [11]:
df

,area,bbox_area,centroid-0,centroid-1,convex_area,eccentricity,equivalent_diameter,euler_number,extent,filled_area,major_axis_length,minor_axis_length,orientation,perimeter,solidity,filename
0,107.0,255.0,9.317757,71.345794,169.0,0.690479,11.672045,1,0.419608,107.0,16.911588,12.233032,0.386300,59.491378,0.633136,40x_Plate8_Slice130_ScWM_Image3_100724_li_thre...
1,139.0,304.0,7.395683,147.503597,203.0,0.850738,13.303394,1,0.457237,139.0,21.235871,11.161348,0.509484,69.905592,0.684729,40x_Plate8_Slice130_ScWM_Image3_100724_li_thre...
2,140.0,228.0,4.614286,214.807143,184.0,0.663624,13.351162,1,0.614035,140.0,17.011030,12.725371,1.197585,65.662951,0.760870,40x_Plate8_Slice130_ScWM_Image3_100724_li_thre...
3,676.0,1720.0,20.449704,286.221893,1105.0,0.656120,29.337858,1,0.393023,676.0,40.256181,30.379587,0.433798,262.214249,0.611765,40x_Plate8_Slice130_ScWM_Image3_100724_li_thre...
4,191.0,275.0,4.408377,322.209424,252.0,0.917122,15.594510,1,0.694545,191.0,26.710149,10.646826,1.544447,82.769553,0.757937,40x_Plate8_Slice130_ScWM_Image3_100724_li_thre...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,113.0,345.0,968.654867,908.504425,221.0,0.874847,11.994835,1,0.327536,113.0,22.024675,10.668723,-0.387489,72.455844,0.511312,40x_Plate3_Slice60_Ctx_Image2_092324_li_thresh...
202,103.0,190.0,993.640777,892.524272,144.0,0.926381,11.451798,1,0.542105,103.0,20.421432,7.690478,-0.166063,59.698485,0.715278,40x_Plate3_Slice60_Ctx_Image2_092324_li_thresh...
203,89.0,225.0,997.662921,5.561798,150.0,0.871994,10.645108,1,0.395556,89.0,19.610547,9.599694,-0.835903,61.662951,0.593333,40x_Plate3_Slice60_Ctx_Image2_092324_li_thresh...
204,92.0,180.0,1016.391304,4.902174,127.0,0.830309,10.823033,1,0.511111,92.0,16.437111,9.160459,-0.779892,52.041631,0.724409,40x_Plate3_Slice60_Ctx_Image2_092324_li_thresh...


*Step 6: Caculating the Circularity*

In [17]:
df['circularity'] = 4*np.pi*df.area/df.perimeter**2

*Step 7: Calculating the Aspect Ratio*

In [18]:
df['aspect_ratio'] = df.major_axis_length/df.minor_axis_length

*Step 8: Add in a column for the ID*

In [19]:
df

,area,bbox_area,centroid-0,centroid-1,convex_area,eccentricity,equivalent_diameter,euler_number,extent,filled_area,major_axis_length,minor_axis_length,orientation,perimeter,solidity,filename,circularity,aspect_ratio
0,107.0,255.0,9.317757,71.345794,169.0,0.690479,11.672045,1,0.419608,107.0,16.911588,12.233032,0.386300,59.491378,0.633136,40x_Plate8_Slice130_ScWM_Image3_100724_li_thre...,0.379914,1.382453
1,139.0,304.0,7.395683,147.503597,203.0,0.850738,13.303394,1,0.457237,139.0,21.235871,11.161348,0.509484,69.905592,0.684729,40x_Plate8_Slice130_ScWM_Image3_100724_li_thre...,0.357438,1.902626
2,140.0,228.0,4.614286,214.807143,184.0,0.663624,13.351162,1,0.614035,140.0,17.011030,12.725371,1.197585,65.662951,0.760870,40x_Plate8_Slice130_ScWM_Image3_100724_li_thre...,0.408035,1.336781
3,676.0,1720.0,20.449704,286.221893,1105.0,0.656120,29.337858,1,0.393023,676.0,40.256181,30.379587,0.433798,262.214249,0.611765,40x_Plate8_Slice130_ScWM_Image3_100724_li_thre...,0.123550,1.325106
4,191.0,275.0,4.408377,322.209424,252.0,0.917122,15.594510,1,0.694545,191.0,26.710149,10.646826,1.544447,82.769553,0.757937,40x_Plate8_Slice130_ScWM_Image3_100724_li_thre...,0.350350,2.508743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,113.0,345.0,968.654867,908.504425,221.0,0.874847,11.994835,1,0.327536,113.0,22.024675,10.668723,-0.387489,72.455844,0.511312,40x_Plate3_Slice60_Ctx_Image2_092324_li_thresh...,0.270484,2.064415
202,103.0,190.0,993.640777,892.524272,144.0,0.926381,11.451798,1,0.542105,103.0,20.421432,7.690478,-0.166063,59.698485,0.715278,40x_Plate3_Slice60_Ctx_Image2_092324_li_thresh...,0.363179,2.655418
203,89.0,225.0,997.662921,5.561798,150.0,0.871994,10.645108,1,0.395556,89.0,19.610547,9.599694,-0.835903,61.662951,0.593333,40x_Plate3_Slice60_Ctx_Image2_092324_li_thresh...,0.294138,2.042831
204,92.0,180.0,1016.391304,4.902174,127.0,0.830309,10.823033,1,0.511111,92.0,16.437111,9.160459,-0.779892,52.041631,0.724409,40x_Plate3_Slice60_Ctx_Image2_092324_li_thresh...,0.426870,1.794354


*Step 10: Saving as a CSV file*

In [22]:
df.to_csv('1_08_2025_li_threshold_all_data.csv')

*Step 11: Print Dependencies and State*

In [23]:
%load_ext watermark

%watermark -v -m -p numpy,pandas,scipy,skimage,matplotlib,wget

%watermark -u -n -t -z

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.20.0

numpy     : 1.26.3
pandas    : 2.2.3
scipy     : 1.11.4
skimage   : 0.22.0
matplotlib: 3.8.2
wget      : not installed

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 24.1.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

Last updated: Wed Jan 08 2025 14:26:49PST

